In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os, sys
from PIL import Image
from tqdm import tqdm

In [2]:
image_dir = '/mnt/analysis/e17023/Adam/10% var PA/images/' # directory of images
output_dir = '/mnt/analysis/e17023/Adam/10% var PA/aug_images/' # directory to save augmented images
image_list = os.listdir(image_dir) # list of images

In [3]:
# generate circle mask for padplane validation
circle_mask = np.ones((145,145))
for i in range(145):
    for j in range(145):
        if math.sqrt((i-72)**2 + (j-72)**2) <= 72:
            circle_mask[i,j] = 0

In [4]:
for i in tqdm(range(len(image_list))):
    image = Image.open(image_dir + image_list[i]) # open image
    image = np.array(image)[:,:,:3] # convert to numpy array
    
    # extract padplane from image
    padplane_bounds = ((3,40),(148,185))
    padplane = image[padplane_bounds[0][0]:padplane_bounds[1][0], padplane_bounds[0][1]:padplane_bounds[1][1], :]
    
    # extract track from padplane
    track = padplane[:,:,0].copy() # copy red channel for track
    track[track == 255] = 0 # set white to black
    track_bounds = np.where(track != 0) # get track bounds
    track_bounds = ((min(track_bounds[0]), max(track_bounds[0])+1), (min(track_bounds[1]), max(track_bounds[1])+1))
    track = track[track_bounds[0][0]:track_bounds[0][1], track_bounds[1][0]:track_bounds[1][1]] # crop track
    
    # place track in random valid location on padplane
    in_bounds = False
    iters = 0
    while not in_bounds:
        padplane[:,:,0] = padplane[:,:,2]; padplane[:,:,1] = padplane[:,:,2] # blank padplane
        loc = (random.randint(0, 36 - (track.shape[0] + 1) // 4), random.randint(0, 36 - (track.shape[1] + 1)//4)) # random location
        
        # insert track into padplane    
        padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,0] += track # red
        padplane[loc[0]*4+1:loc[0]*4+track.shape[0]+1, loc[1]*4+1:loc[1]*4+track.shape[1]+1,1] += track # green
        
        # test for track within radius of padplane
        test_padplane = padplane.copy()
        test_padplane *= circle_mask[:,:,np.newaxis].astype(padplane.dtype)
        if np.sum(test_padplane[:,:,0]) == np.sum(test_padplane[:,:,2]):
            in_bounds = True
        else: # track is outside radius, try again
            iters += 1
            if iters > 100:
                print('too many iterations')
                break
    
    # randomly rotate padplane
    padplane = np.rot90(padplane, random.randint(0,3), (0,1)) 
    
    # place padplane back into image
    image[padplane_bounds[0][0]:padplane_bounds[1][0], padplane_bounds[0][1]:padplane_bounds[1][1], :] = padplane
    
    # save image
    plt.imsave(output_dir + image_list[i], image)

  0%|          | 0/19027 [00:00<?, ?it/s]

100%|██████████| 19027/19027 [05:39<00:00, 56.06it/s]
